In [1]:

from random import shuffle
from random import shuffle
import glob
import numpy as np
import h5py
import cv2

In [33]:
#hdf5_path = 'C:/Users/ateya/OneDrive/Desktop/Graduate Research/RecentUpdate/bloodataset/blooddata1.hdf5'  # address to where you want to save the hdf5 file
hdf5_path = './blooddata-color.hdf5'
img_width = 100
img_height = 100

In [34]:
shuffle_data = True  # shuffle the addresses before saving
#train_path = './train_images_rotated/*.jpg'

#test_path = './test_images_rotated/*.jpg'
image_path = './images/*.jpg'
# read addresses and labels from the 'train' folder
addrs = glob.glob(image_path)
#testAddrs = glob.glob(test_path)

def createLabels(addrs):
    labels = []
    for addr in addrs:
        if '10_' in addr:
            labels.append(10)
        elif '20_' in addr:
            labels.append(20)
        elif '30_' in addr:
            labels.append(30)
        elif '40_' in addr:
            labels.append(40)
        elif '50_' in addr:
            labels.append(50)
        elif '60_' in addr:
            labels.append(60)
        elif '70_' in addr:
            labels.append(70)
        elif '80_' in addr:
            labels.append(70)
        elif '90_' in addr:
            labels.append(70)
    return labels

labels = createLabels(addrs)
#testLabels = createLabels(testAddrs)
addrs = np.string_(addrs)
#testAddrs = np.string_(testAddrs)


 
            
#labels = [10 if '10' in addr else 30 for addr in addrs]  # 10 = 10, 30 = 30

# to shuffle data
if shuffle_data:
    c = list(zip(addrs, labels))
    shuffle(c)
    addrs, labels = zip(*c)
    
# Divide the data into 80% train, 10% validation, and 10% test
#train_addrs = addrs[0:int(0.7*len(addrs))]
#train_labels = labels[0:int(0.7*len(labels))]
#test_addrs = addrs[int(0.7*len(addrs)):]
#test_labels = labels[int(0.7*len(labels)):]

'''Uncomment to use separate folder for test images'''
test_addrs = addrs[int(0.9*len(addrs)):]
test_labels = labels[int(0.9*len(labels)):]

dev_addrs = addrs[int(0.8*len(addrs)):int(0.9*len(addrs))]
dev_labels = labels[int(0.8*len(labels)):int(0.9*len(labels))]

train_addrs = addrs[0:int(0.8*len(addrs))]
train_labels = labels[0:int(0.8*len(labels))]

In [37]:
len(train_addrs)

16300

In [38]:
train_shape = (len(train_addrs), img_height, img_width, 3)
test_shape = (len(test_addrs), img_height, img_width, 3)
dev_shape = (len(dev_addrs), img_height, img_width, 3) # New

# open a hdf5 file and create earrays
hdf5_file = h5py.File(hdf5_path, mode='w')
hdf5_file.create_dataset("train_imgs_resized", train_shape, np.int8)
hdf5_file.create_dataset("test_imgs_resized", test_shape, np.int8)
hdf5_file.create_dataset("dev_imgs_resized", dev_shape, np.int8) #new

#hdf5_file.create_dataset("train_mean", train_shape[1:], np.float32)
hdf5_file.create_dataset("train_labels", (len(train_addrs),), np.int8)
hdf5_file["train_labels"][...] = train_labels
hdf5_file.create_dataset("train_addrs", (len(train_addrs),), dtype="S20")
hdf5_file["train_addrs"][...] = train_addrs
hdf5_file.create_dataset("test_addrs", (len(test_addrs),), dtype="S20")
hdf5_file["test_addrs"][...] = test_addrs
hdf5_file.create_dataset("test_labels", (len(test_addrs),), np.int8)
hdf5_file["test_labels"][...] = test_labels
#New 
hdf5_file.create_dataset("dev_addrs", (len(dev_addrs),), dtype="S20")
hdf5_file["dev_addrs"][...] = dev_addrs
hdf5_file.create_dataset("dev_labels", (len(dev_addrs),), np.int8)
hdf5_file["dev_labels"][...] = dev_labels

In [39]:
# a numpy array to save the mean of the images
mean = np.zeros(train_shape[1:], np.float32)
# loop over train addresses
for i in range(len(train_addrs)):
   
    # read an image 
    addr = train_addrs[i]
    #print(str(addr))
    
    # cv2 load images
    img = cv2.imread(addr.decode('utf-8'))
    
    # save the image and calculate the mean so far
    hdf5_file["train_imgs_resized"][i, ...] = img[None]
    mean += img / float(len(train_labels))

# loop over test addresses
for i in range(len(test_addrs)):
   
    # read an image 
    # cv2 load images
    addr = test_addrs[i]
    img = cv2.imread(addr.decode('utf-8'))
    
    # save the image
    hdf5_file["test_imgs_resized"][i, ...] = img[None]

# loop over dev addresses
for i in range(len(dev_addrs)):
   
    # read an image 
    # cv2 load images
    addr = dev_addrs[i]
    img = cv2.imread(addr.decode('utf-8'))
    
    # save the image
    hdf5_file["dev_imgs_resized"][i, ...] = img[None]
# save the mean and close the hdf5 file
#hdf5_file["train_mean"][...] = mean


In [41]:
hdf5_file.close()